In [1]:
import pandas as pd
import numpy as np

In [2]:
data_df = pd.read_csv('Data.csv')
data_df.head()

,Date,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
0,2014/9,-0.017895,-0.153914,-0.020975,-0.012117,-0.044718,-0.010923,-0.111896,-0.041635,-0.012791,-0.066980,0.002030,0.002623,0.001433,0.000014
1,2014/10,0.037613,-0.118071,0.009492,0.006239,-0.035197,-0.103075,-0.059466,0.001171,0.043103,0.081262,0.001953,0.002536,0.001363,0.000007
2,2014/11,0.026912,0.160547,-0.012890,0.011114,-0.000357,-0.140457,-0.041990,-0.003123,0.042380,0.002836,0.001873,0.002457,0.001301,0.000003
3,2014/12,-0.001069,-0.155709,-0.002227,0.000587,-0.025733,-0.226444,-0.045627,-0.037137,-0.014884,0.041351,0.001810,0.002342,0.001313,0.000018
4,2015/1,-0.029110,-0.307988,0.007031,0.033881,0.082135,-0.104072,0.094639,-0.060877,-0.017875,-0.027159,0.001569,0.002039,0.001161,0.000008


In [3]:
testing_df = data_df.loc[data_df['Date'] > '2019'].drop(columns='Date')

testing_df.head()

,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
52,0.078943,-0.100358,0.050871,0.030485,0.028188,0.152284,0.033654,0.000591,0.085744,0.106936,0.002180,0.002461,0.002037,0.001940
53,0.031919,0.105174,0.012636,0.000345,-0.004096,0.027515,-0.019437,-0.028943,0.034279,0.050945,0.002223,0.002513,0.002067,0.001947
54,0.007346,0.063691,0.010046,0.031090,0.001067,0.072041,-0.001408,0.001216,0.030220,-0.032546,0.002129,0.002444,0.001978,0.001940
55,0.028653,0.286794,0.010108,0.008118,-0.002715,0.036661,-0.009187,-0.033113,0.041200,0.022810,0.002062,0.002397,0.001894,0.001934
56,-0.056681,0.587078,-0.016108,0.016831,0.024336,-0.160363,-0.007273,0.033582,-0.079289,-0.069711,0.001918,0.002263,0.001744,0.001920


In [4]:
testing_df.tail()

,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
107,-0.013429,-0.126172,0.006821,-0.002349,-0.002438,0.028783,0.004486,-0.010018,-0.012540,-0.046461,0.003332,0.003398,0.003472,0.004296
108,-0.052550,0.045239,-0.017983,-0.028658,-0.048082,0.051222,-0.082544,-0.016988,-0.051167,-0.073932,0.003573,0.003673,0.003634,0.004300
109,-0.021319,0.238854,-0.004390,-0.015096,0.085116,-0.055045,0.080330,0.019926,-0.028760,-0.054863,0.003898,0.004001,0.003888,0.004327
110,0.079828,0.064212,0.038973,0.060567,0.028554,-0.061672,0.103004,0.013649,0.089282,0.085821,0.003731,0.003870,0.003676,0.004288
111,0.035308,0.092441,0.018423,0.032565,-0.004375,-0.037406,-0.066438,-0.058129,0.050213,0.085447,0.003311,0.003449,0.003272,0.004231


In [5]:
np.array(testing_df.mean())

array([0.01001564, 0.05285495, 0.00253649, 0.00168611, 0.00500961,
       0.0051547 , 0.00458695, 0.00418095, 0.0156151 , 0.00635026,
       0.0018521 , 0.00218611, 0.0016922 , 0.00153542])

# MVO

In [69]:
import numpy as np
from scipy.optimize import minimize

# Define inputs
returns = np.array(testing_df.mean())  # Expected returns vector
cov_matrix = np.array(testing_df.cov())  # Covariance matrix

target_return = 0.01

# Objective: Minimize portfolio variance
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# Constraints
def weight_constraint(weights):
    return np.sum(weights) - 1

def return_constraint(weights):
    return weights @ returns - target_return

# Initial guess (equal weighting)
num_assets = len(returns)
init_guess = np.array([1 / num_assets] * num_assets)

# Set constraints and bounds
constraints = ({'type': 'eq', 'fun': weight_constraint},
              {'type': 'eq', 'fun': return_constraint})
bounds = [(0, 1) for _ in range(num_assets)]

# Optimize for minimum variance
result = minimize(portfolio_variance, init_guess, args=(cov_matrix,), method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x


In [70]:
result

     fun: 0.0009005991244978219
     jac: array([ 2.27056848e-03,  1.13546415e-02,  1.12778295e-03,  9.79153636e-04,
        8.61132336e-04,  8.92928765e-04,  2.30499434e-03,  7.14955866e-04,
        2.73807968e-03,  2.80991855e-03, -5.23158087e-06, -2.48951255e-06,
       -7.50145409e-06,  1.84916280e-06])
 message: 'Optimization terminated successfully'
    nfev: 225
     nit: 15
    njev: 15
  status: 0
 success: True
       x: array([0.        , 0.11122843, 0.        , 0.        , 0.09351648,
       0.00823104, 0.        , 0.08711355, 0.14242715, 0.        ,
       0.14109381, 0.15373425, 0.13522834, 0.12742695])

In [71]:
weights_df = pd.DataFrame(optimal_weights.tolist()).transpose()

weights_df.columns = testing_df.columns

weights_df

,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
0,0.0,0.111228,0.0,0.0,0.093516,0.008231,0.0,0.087114,0.142427,0.0,0.141094,0.153734,0.135228,0.127427


In [72]:
# Calculate portfolio's expected return and variance
portfolio_return = optimal_weights @ returns  # Portfolio expected return
portfolio_variance_value = optimal_weights.T @ cov_matrix @ optimal_weights  # Portfolio variance

risk_free_rate = 0.003

# Calculate lambda
lambda_value = (portfolio_return - risk_free_rate) / portfolio_variance_value

print("Risk aversion parameter (lambda):", lambda_value)


Risk aversion parameter (lambda): 7.772603604398487


# Geometric

In [65]:
import numpy as np

# Example monthly returns for each asset
returns_data = {
    'Asset_A': [0.02, 0.03, -0.01, 0.04, 0.01],
    'Asset_B': [0.01, -0.02, 0.03, 0.05, 0.02],
    'Asset_C': [0.005, 0.004, 0.006, 0.003, 0.004]
}

# Calculate monthly geometric mean for each asset
monthly_geometric_means = {}
for asset in testing_df.columns:
    
    product = np.prod([1 + r for r in testing_df[asset]])  # Multiply (1 + return) for each month
    monthly_geometric_mean = (product ** (1 / len(returns))) - 1
    monthly_geometric_means[asset] = monthly_geometric_mean

print("Monthly Geometric Means:", monthly_geometric_means)


Monthly Geometric Means: {'SPY': 0.03655752218150221, 'BTC-USD': 0.15220071555350057, 'HYG': 0.0090951258647618, 'LQD': 0.00542608827769131, 'GLD': 0.01814244961787881, 'USO': -0.025535818006474864, 'SLV': 0.0030000581927085435, 'DBA': 0.015883438167124675, 'QQQ': 0.05835798131082126, 'IWM': 0.015685597628068537, '^TNX': 0.007959753596717523, '^TYX': 0.009401419759097474, '^FVX': 0.007269661105182124, '^IRX': 0.006591770939105013}


In [66]:
import numpy as np
from scipy.optimize import minimize

# Define inputs
returns = np.array(list(monthly_geometric_means.values()))  # Expected returns vector
cov_matrix = np.array(testing_df.cov())  # Covariance matrix

target_return = 0.01

# Objective: Minimize portfolio variance
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# Constraints
def weight_constraint(weights):
    return np.sum(weights) - 1

def return_constraint(weights):
    return weights @ returns - target_return

# Initial guess (equal weighting)
num_assets = len(returns)
init_guess = np.array([1 / num_assets] * num_assets)

# Set constraints and bounds
constraints = ({'type': 'eq', 'fun': weight_constraint},
              {'type': 'eq', 'fun': return_constraint})
bounds = [(0, 1) for _ in range(num_assets)]

# Optimize for minimum variance
result = minimize(portfolio_variance, init_guess, args=(cov_matrix,), method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x
result

     fun: 8.58862096215662e-05
     jac: array([7.96640954e-04, 1.68569134e-03, 4.55113657e-04, 4.37436620e-04,
       5.18863813e-04, 5.43825552e-04, 1.16570651e-03, 2.78311712e-04,
       9.00659148e-04, 8.74674331e-04, 2.81681423e-07, 5.83429937e-07,
       6.86632120e-08, 2.73889054e-06])
 message: 'Optimization terminated successfully'
    nfev: 120
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([1.04083409e-17, 2.37531369e-03, 8.80468143e-02, 1.06991353e-01,
       9.26110379e-02, 3.03576608e-18, 6.93889390e-18, 1.02264138e-01,
       4.24617958e-03, 5.20417043e-18, 1.50695047e-01, 1.48769696e-01,
       1.51640444e-01, 1.52359977e-01])

In [67]:
weights_df = pd.DataFrame(optimal_weights.tolist()).transpose()

weights_df.columns = testing_df.columns

weights_df

,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
0,1.040834e-17,0.002375,0.088047,0.106991,0.092611,3.035766e-18,6.938894e-18,0.102264,0.004246,5.204170e-18,0.150695,0.14877,0.15164,0.15236


### Risk Aversion lambda with target gain

In [68]:
# Calculate portfolio's expected return and variance
portfolio_return = optimal_weights @ returns  # Portfolio expected return
portfolio_variance_value = optimal_weights.T @ cov_matrix @ optimal_weights  # Portfolio variance

risk_free_rate = 0.003

# Calculate lambda
lambda_value = (portfolio_return - risk_free_rate) / portfolio_variance_value

print("Risk aversion parameter (lambda):", lambda_value)


Risk aversion parameter (lambda): 81.50318929346685


# Sharpe Ratio

In [32]:
import numpy as np
from scipy.optimize import minimize

# Define inputs
returns = np.array(list(monthly_geometric_means.values()))  # Expected monthly returns for each asset
cov_matrix = np.array(testing_df.cov()) # Monthly covariance matrix

risk_free_rate = 0.003  # Monthly risk-free rate - 0.3%

# Objective: Maximize Sharpe Ratio by minimizing negative Sharpe Ratio
def neg_sharpe_ratio(weights, returns, cov_matrix, risk_free_rate):
    # Calculate portfolio return and standard deviation
    port_return = weights @ returns
    port_variance = weights.T @ cov_matrix @ weights
    port_std_dev = np.sqrt(port_variance)
    
    # Calculate Sharpe Ratio
    sharpe_ratio = (port_return - risk_free_rate) / port_std_dev
    return -sharpe_ratio  # Negative Sharpe Ratio for minimization

# Constraints
def weight_constraint(weights):
    return np.sum(weights) - 1  # Weights must sum to 1

# Initial guess (equal weighting)
num_assets = len(returns)
init_guess = np.array([1 / num_assets] * num_assets)

# Set constraints and bounds
constraints = [{'type': 'eq', 'fun': weight_constraint}]  # Sum of weights must equal 1
bounds = [(0, 1) for _ in range(num_assets)]  # Weights between 0 and 1

# Optimize to maximize the Sharpe Ratio
result = minimize(neg_sharpe_ratio, init_guess, args=(returns, cov_matrix, risk_free_rate), 
                  method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x
print("Optimal weights for max Sharpe ratio:", optimal_weights)

# Calculated optimal portfolio return, variance, and Sharpe ratio
optimal_return = optimal_weights @ returns
optimal_variance = optimal_weights.T @ cov_matrix @ optimal_weights
optimal_std_dev = np.sqrt(optimal_variance)
optimal_sharpe_ratio = (optimal_return - risk_free_rate) / optimal_std_dev

print("Optimal Portfolio Return:", optimal_return)
print("Optimal Portfolio Standard Deviation:", optimal_std_dev)
print("Optimal Sharpe Ratio:", optimal_sharpe_ratio)


Optimal weights for max Sharpe ratio: [2.44770106e-09 3.10097442e-04 0.00000000e+00 1.47378207e-10
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.35039208e-03
 1.03349616e-03 1.26923367e-09 0.00000000e+00 9.95306183e-01
 0.00000000e+00 0.00000000e+00]
Optimal Portfolio Return: 0.009518016872603001
Optimal Portfolio Standard Deviation: 0.0007861942829538919
Optimal Sharpe Ratio: 8.290593068310654


In [33]:
weights_df = pd.DataFrame(optimal_weights.tolist()).transpose()

weights_df.columns = testing_df.columns

weights_df

,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
0,2.447701e-09,0.00031,0.0,1.473782e-10,0.0,0.0,0.0,0.00335,0.001033,1.269234e-09,0.0,0.995306,0.0,0.0


# Risk Aversion

In [14]:
import numpy as np

# Example monthly returns for each asset
returns_data = {
    'Asset_A': [0.02, 0.03, -0.01, 0.04, 0.01],
    'Asset_B': [0.01, -0.02, 0.03, 0.05, 0.02],
    'Asset_C': [0.005, 0.004, 0.006, 0.003, 0.004]
}

# Calculate monthly geometric mean for each asset
monthly_geometric_means = {}
for asset in testing_df.columns:
    
    product = np.prod([1 + r for r in testing_df[asset]])  # Multiply (1 + return) for each month
    monthly_geometric_mean = (product ** (1 / len(returns))) - 1
    monthly_geometric_means[asset] = monthly_geometric_mean

print("Monthly Geometric Means:", monthly_geometric_means)


Monthly Geometric Means: {'SPY': 0.03655752218150221, 'BTC-USD': 0.15220071555350057, 'HYG': 0.0090951258647618, 'LQD': 0.00542608827769131, 'GLD': 0.01814244961787881, 'USO': -0.025535818006474864, 'SLV': 0.0030000581927085435, 'DBA': 0.015883438167124675, 'QQQ': 0.05835798131082126, 'IWM': 0.015685597628068537, '^TNX': 0.007959753596717523, '^TYX': 0.009401419759097474, '^FVX': 0.007269661105182124, '^IRX': 0.006591770939105013}


In [15]:
import numpy as np
from scipy.optimize import minimize

# Step 1: Define Inputs
# Define inputs
returns = np.array(list(monthly_geometric_means.values()))  # Expected returns vector
cov_matrix = np.array(testing_df.cov())  # Covariance matrix

risk_aversion = 10  # Risk aversion coefficient (λ)

# Step 2: Define Utility Function
def utility_function(weights, returns, cov_matrix, risk_aversion):
    # Calculate portfolio expected return
    port_return = weights @ returns
    
    # Calculate portfolio variance
    port_variance = weights.T @ cov_matrix @ weights
    
    # Utility = Expected Return - (λ/2) * Variance
    utility = port_return - (risk_aversion / 2) * port_variance
    return -utility  # Negative for maximization

# Step 3: Define Constraints and Initial Guess
def weight_constraint(weights):
    return np.sum(weights) - 1  # Weights must sum to 1

num_assets = len(returns)
init_guess = np.array([1 / num_assets] * num_assets)

# Constraints and bounds
constraints = [{'type': 'eq', 'fun': weight_constraint}]
bounds = [(0, 1) for _ in range(num_assets)]

# Step 4: Optimize for Maximum Utility
result = minimize(utility_function, init_guess, args=(returns, cov_matrix, risk_aversion), 
                  method='SLSQP', bounds=bounds, constraints=constraints)

# Extract optimal weights and display results
optimal_weights = result.x
print("Optimal weights to maximize utility:", optimal_weights)

# Calculate expected return, variance, and utility of the optimized portfolio
optimal_return = optimal_weights @ returns
optimal_variance = optimal_weights.T @ cov_matrix @ optimal_weights
optimal_utility = optimal_return - (risk_aversion / 2) * optimal_variance

print("Optimal Portfolio Expected Return:", optimal_return)
print("Optimal Portfolio Variance:", optimal_variance)
print("Optimal Portfolio Utility:", optimal_utility)


Optimal weights to maximize utility: [1.05626905e-16 2.46039755e-01 3.06540750e-17 0.00000000e+00
 5.89805982e-17 1.68007765e-17 0.00000000e+00 0.00000000e+00
 7.53960245e-01 0.00000000e+00 0.00000000e+00 1.08473526e-17
 0.00000000e+00 0.00000000e+00]
Optimal Portfolio Expected Return: 0.08144702469701064
Optimal Portfolio Variance: 0.007126561703951992
Optimal Portfolio Utility: 0.04581421617725068


In [16]:
weights_df = pd.DataFrame(optimal_weights.tolist()).transpose()

weights_df.columns = testing_df.columns

weights_df

,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
0,1.056269e-16,0.24604,3.065408e-17,0.0,5.898060e-17,1.680078e-17,0.0,0.0,0.75396,0.0,0.0,1.084735e-17,0.0,0.0


# Combine

## Geometric

In [17]:
import numpy as np
from scipy.optimize import minimize

# Define inputs
expected_returns = np.array(list(monthly_geometric_means.values())) # Expected returns for each asset
cov_matrix = np.array(testing_df.cov())  # Covariance matrix

risk_free_rate = 0.003  # Risk-free rate
lambda_value = 3  # Risk aversion coefficient

# Sharpe Ratio Function
# Objective: Maximize Sharpe Ratio by minimizing negative Sharpe Ratio
def sharpe_ratio(weights, returns, cov_matrix, risk_free_rate):
    # Calculate portfolio return and standard deviation
    port_return = weights @ returns
    port_variance = weights.T @ cov_matrix @ weights
    port_std_dev = np.sqrt(port_variance)
    
    # Calculate Sharpe Ratio
    sharpe_ratio = (port_return - risk_free_rate) / port_std_dev
    return sharpe_ratio 


# Utility Function
def utility_function(weights, returns, cov_matrix, risk_aversion):
    # Calculate portfolio expected return
    port_return = weights @ returns
    
    # Calculate portfolio variance
    port_variance = weights.T @ cov_matrix @ weights
    
    # Utility = Expected Return - (λ/2) * Variance
    utility = port_return - (risk_aversion / 2) * port_variance
    return utility 


# Step 3: Define the Combined Objective Function
def combined_objective(weights, expected_returns, cov_matrix, risk_free_rate, lambda_value, w1=0.4, w2=0.6):
    # Maximize Sharpe Ratio + Utility (weighted)
    sharpe = sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate)
    utility = utility_function(weights, expected_returns, cov_matrix, lambda_value)
    return -(w1 * sharpe + w2 * utility)  # Minimize the negative of the combined function

# Step 4: Set Constraints and Bounds
num_assets = len(expected_returns)
init_guess = np.array([1 / num_assets] * num_assets)

constraints = [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}]  # Weights must sum to 1
bounds = [(0, 1) for _ in range(num_assets)]  # No short selling

# Step 5: Optimize the Combined Function
result = minimize(combined_objective, init_guess, args=(expected_returns, cov_matrix, risk_free_rate, lambda_value),
                  method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x
print("Optimal Weights for Combined Sharpe + Utility:", optimal_weights)


Optimal Weights for Combined Sharpe + Utility: [4.46824699e-13 3.09074680e-04 8.65313541e-13 2.95760429e-13
 6.83186907e-13 2.93427704e-12 5.65560568e-13 3.35248885e-03
 1.03519555e-03 9.24664793e-13 4.32699114e-13 9.95303241e-01
 1.74467647e-12 1.19153814e-12]


In [18]:
weights_df = pd.DataFrame(optimal_weights.tolist()).transpose()

weights_df.columns = testing_df.columns

weights_df

,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
0,4.468247e-13,0.000309,8.653135e-13,2.957604e-13,6.831869e-13,2.934277e-12,5.655606e-13,0.003352,0.001035,9.246648e-13,4.326991e-13,0.995303,1.744676e-12,1.191538e-12


## Mean

In [19]:
import numpy as np
from scipy.optimize import minimize

# Define inputs
expected_returns = np.array(testing_df.mean()) # Expected returns for each asset
cov_matrix = np.array(testing_df.cov())  # Covariance matrix

risk_free_rate = 0.003  # Risk-free rate
lambda_value = 3  # Risk aversion coefficient

# Sharpe Ratio Function
# Objective: Maximize Sharpe Ratio by minimizing negative Sharpe Ratio
def sharpe_ratio(weights, returns, cov_matrix, risk_free_rate):
    # Calculate portfolio return and standard deviation
    port_return = weights @ returns
    port_variance = weights.T @ cov_matrix @ weights
    port_std_dev = np.sqrt(port_variance)
    
    # Calculate Sharpe Ratio
    sharpe_ratio = (port_return - risk_free_rate) / port_std_dev
    return sharpe_ratio 


# Utility Function
def utility_function(weights, returns, cov_matrix, risk_aversion):
    # Calculate portfolio expected return
    port_return = weights @ returns
    
    # Calculate portfolio variance
    port_variance = weights.T @ cov_matrix @ weights
    
    # Utility = Expected Return - (λ/2) * Variance
    utility = port_return - (risk_aversion / 2) * port_variance
    return utility 


# Step 3: Define the Combined Objective Function
def combined_objective(weights, expected_returns, cov_matrix, risk_free_rate, lambda_value, w1=0.4, w2=0.6):
    # Maximize Sharpe Ratio + Utility (weighted)
    sharpe = sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate)
    utility = utility_function(weights, expected_returns, cov_matrix, lambda_value)
    return -(w1 * sharpe + w2 * utility)  # Minimize the negative of the combined function

# Step 4: Set Constraints and Bounds
num_assets = len(expected_returns)
init_guess = np.array([1 / num_assets] * num_assets)

constraints = [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}]  # Weights must sum to 1
bounds = [(0, 1) for _ in range(num_assets)]  # No short selling

# Step 5: Optimize the Combined Function
result = minimize(combined_objective, init_guess, args=(expected_returns, cov_matrix, risk_free_rate, lambda_value),
                  method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x
print("Optimal Weights for Combined Sharpe + Utility:", optimal_weights)


Optimal Weights for Combined Sharpe + Utility: [3.36524060e-17 3.69258194e-01 1.63923918e-17 0.00000000e+00
 1.27794054e-16 6.98872937e-03 0.00000000e+00 0.00000000e+00
 6.23753077e-01 0.00000000e+00 0.00000000e+00 1.00183395e-17
 1.32979216e-17 5.72428845e-18]


In [20]:
weights_df = pd.DataFrame(optimal_weights.tolist()).transpose()

weights_df.columns = testing_df.columns

weights_df

,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
0,3.365241e-17,0.369258,1.639239e-17,0.0,1.277941e-16,0.006989,0.0,0.0,0.623753,0.0,0.0,1.001834e-17,1.329792e-17,5.724288e-18


# Maximum Decorrelation

In [21]:
import numpy as np
from scipy.optimize import minimize

# Define inputs
expected_returns = np.array(testing_df.mean()) # Expected returns for each asset
cov_matrix = np.array(testing_df.cov())  # Covariance matrix

risk_free_rate = 0.003  # Risk-free rate
lambda_value = 3  # Risk aversion coefficient

alpha = 0.1

# Sharpe Ratio Function
# Objective: Maximize Sharpe Ratio by minimizing negative Sharpe Ratio
def sharpe_ratio(weights, returns, cov_matrix, risk_free_rate):
    # Calculate portfolio return and standard deviation
    port_return = weights @ returns
    port_variance = weights.T @ cov_matrix @ weights
    port_std_dev = np.sqrt(port_variance)
    
    # Calculate Sharpe Ratio
    sharpe_ratio = (port_return - risk_free_rate) / port_std_dev
    return sharpe_ratio 


# Utility Function
def utility_function(weights, returns, cov_matrix, risk_aversion):
    # Calculate portfolio expected return
    port_return = weights @ returns
    
    # Calculate portfolio variance
    port_variance = weights.T @ cov_matrix @ weights
    
    # Utility = Expected Return - (λ/2) * Variance
    utility = port_return - (risk_aversion / 2) * port_variance
    return utility 


# Step 3: Define the Combined Objective Function
def combined_objective(weights, expected_returns, cov_matrix, risk_free_rate, lambda_value, w1=0.4, w2=0.6):
    # Maximize Sharpe Ratio + Utility (weighted)
    sharpe = sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate)
    utility = utility_function(weights, expected_returns, cov_matrix, lambda_value)
    
    penalty = alpha * (weights.T @ cov_matrix @weights) 
    return -(w1 * sharpe + w2 * utility + penalty)  # Minimize the negative of the combined function

# Step 4: Set Constraints and Bounds
num_assets = len(expected_returns)
init_guess = np.array([1 / num_assets] * num_assets)

constraints = [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}]  # Weights must sum to 1
bounds = [(0, 1) for _ in range(num_assets)]  # No short selling

# Step 5: Optimize the Combined Function
result = minimize(combined_objective, init_guess, args=(expected_returns, cov_matrix, risk_free_rate, lambda_value),
                  method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x
print("Optimal Weights for Combined Sharpe + Utility:", optimal_weights)


Optimal Weights for Combined Sharpe + Utility: [1.04731333e-17 3.79389576e-01 0.00000000e+00 2.19991432e-17
 8.93030517e-17 2.80656908e-03 4.30914950e-18 0.00000000e+00
 6.17803855e-01 1.15111694e-17 0.00000000e+00 0.00000000e+00
 4.11387575e-18 0.00000000e+00]


In [22]:
weights_df = pd.DataFrame(optimal_weights.tolist()).transpose()

weights_df.columns = testing_df.columns

weights_df

,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
0,1.047313e-17,0.37939,0.0,2.199914e-17,8.930305e-17,0.002807,4.309150e-18,0.0,0.617804,1.151117e-17,0.0,0.0,4.113876e-18,0.0


# Maximum Diversification

In [23]:
import numpy as np
from scipy.optimize import minimize

# Define inputs
expected_returns = np.array(testing_df.mean()) # Expected returns for each asset
cov_matrix = np.array(testing_df.cov())  # Covariance matrix

risk_free_rate = 0.003  # Risk-free rate
lambda_value = 3  # Risk aversion coefficient

alpha = 0.3

# Sharpe Ratio Function
# Objective: Maximize Sharpe Ratio by minimizing negative Sharpe Ratio
def sharpe_ratio(weights, returns, cov_matrix, risk_free_rate):
    # Calculate portfolio return and standard deviation
    port_return = weights @ returns
    port_variance = weights.T @ cov_matrix @ weights
    port_std_dev = np.sqrt(port_variance)
    
    # Calculate Sharpe Ratio
    sharpe_ratio = (port_return - risk_free_rate) / port_std_dev
    return sharpe_ratio 


# Utility Function
def utility_function(weights, returns, cov_matrix, risk_aversion):
    # Calculate portfolio expected return
    port_return = weights @ returns
    
    # Calculate portfolio variance
    port_variance = weights.T @ cov_matrix @ weights
    
    # Utility = Expected Return - (λ/2) * Variance
    utility = port_return - (risk_aversion / 2) * port_variance
    return utility 

# Diversification Ratio (DR)
def diversification_ratio(weights, cov_matrix):
    port_std = np.sqrt(weights.T @ cov_matrix @ weights)
    weighted_std = np.sum(weights * np.sqrt(np.diagonal(cov_matrix)))
    return weighted_std / port_std


# Step 3: Define the Combined Objective Function
def combined_objective(weights, expected_returns, cov_matrix, risk_free_rate, lambda_value, w1=0.4, w2=0.4, w3=0.2):
    # Maximize Sharpe Ratio + Utility (weighted)
    sharpe = sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate)
    utility = utility_function(weights, expected_returns, cov_matrix, lambda_value)
    diverse = diversification_ratio(weights, cov_matrix)
    
    penalty = alpha * (weights.T @ cov_matrix @weights) 
    return -(w1 * sharpe + w2 * utility + w3 * diverse)  # Minimize the negative of the combined function

# Step 4: Set Constraints and Bounds
num_assets = len(expected_returns)
init_guess = np.array([1 / num_assets] * num_assets)

constraints = [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}]  # Weights must sum to 1
bounds = [(0, 1) for _ in range(num_assets)]  # No short selling

# Step 5: Optimize the Combined Function
result = minimize(combined_objective, init_guess, args=(expected_returns, cov_matrix, risk_free_rate, lambda_value),
                  method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x
print("Optimal Weights for Combined Sharpe + Utility:", optimal_weights)


Optimal Weights for Combined Sharpe + Utility: [3.10423695e-17 1.07488960e-01 0.00000000e+00 2.24511130e-17
 4.11448472e-01 1.38413019e-01 0.00000000e+00 1.79079583e-01
 1.63569965e-01 6.78347161e-17 2.09153085e-18 0.00000000e+00
 6.59439945e-18 0.00000000e+00]


In [24]:
weights_df = pd.DataFrame(optimal_weights.tolist()).transpose()

weights_df.columns = testing_df.columns

weights_df

,SPY,BTC-USD,HYG,LQD,GLD,USO,SLV,DBA,QQQ,IWM,^TNX,^TYX,^FVX,^IRX
0,3.104237e-17,0.107489,0.0,2.245111e-17,0.411448,0.138413,0.0,0.17908,0.16357,6.783472e-17,2.091531e-18,0.0,6.594399e-18,0.0
